In [1]:
import pandas as pd


In [16]:
Weather_full = pd.read_csv("./Wetterdaten_Basel_stündlich22_24.csv", skiprows=9)

In [17]:
Weather_full.head()


,timestamp,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
0,20220101T0000,5.560245,0.0,4.320000,180.00000
1,20220101T0100,4.630245,0.0,3.259939,173.65980
2,20220101T0200,3.660245,0.0,3.545589,156.03750
3,20220101T0300,3.880245,0.0,3.671294,168.69006
4,20220101T0400,3.650245,0.0,4.349896,155.55605


In [18]:
# Rename the column
Weather_full.rename(columns={'timestamp': 'Datum'}, inplace=True)

# Change the format of the 'Datum' column
Weather_full['Datum'] = pd.to_datetime(Weather_full['Datum'], format='%Y%m%dT%H%M', utc=True)

# Sort by 'Datum' and set it as the index
Weather_full.sort_values(by='Datum', inplace=True)
Weather_full.set_index('Datum', inplace=True)

# Display the first few rows to verify the changes
Weather_full.head()

,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
Datum,,,,
2022-01-01 00:00:00+00:00,5.560245,0.0,4.320000,180.00000
2022-01-01 01:00:00+00:00,4.630245,0.0,3.259939,173.65980
2022-01-01 02:00:00+00:00,3.660245,0.0,3.545589,156.03750
2022-01-01 03:00:00+00:00,3.880245,0.0,3.671294,168.69006
2022-01-01 04:00:00+00:00,3.650245,0.0,4.349896,155.55605


import numpy as np
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

target_columns = ["Basel Temperature [2 m elevation corrected]", "Basel Precipitation Total", "Basel Wind Speed [10 m]", "Basel Wind Direction [10 m]"]

for col in target_columns:
    Weather_full[col] = scaler.fit_transform(Weather_full[col].values.reshape(-1, 1))


means = Weather_full[target_columns].mean()
stds = Weather_full[target_columns].std()

print(f"Means: {means}")

print(f"Stds: {stds}")

Weather_full.head()

In [19]:
#hopefully useless
#checking for missing values
Weather_full.reset_index(inplace=True)

# Create a date range from the minimum to the maximum date in the 'Datum' column
date_range = pd.date_range(start=Weather_full['Datum'].min(), end=Weather_full['Datum'].max(), freq='D')

# Create a DataFrame with all the dates in the range
all_dates = pd.DataFrame({'Datum': date_range})

# Merge this new DataFrame with the original to find missing dates
missing_dates = pd.merge(all_dates, Weather_full, on='Datum', how='left', indicator=True)

# Rows that are missing in the original DataFrame will have '_merge' == 'left_only'
missing_rows = missing_dates[missing_dates['_merge'] == 'left_only']

# Output the missing rows (if any)
if not missing_rows.empty:
    print(f"Missing rows for the following dates: {missing_rows['Datum'].tolist()}")
else:
    print("No rows are missing.")

No rows are missing.


In [20]:
Weather_full.head()

,Datum,Basel Temperature [2 m elevation corrected],Basel Precipitation Total,Basel Wind Speed [10 m],Basel Wind Direction [10 m]
0,2022-01-01 00:00:00+00:00,5.560245,0.0,4.320000,180.00000
1,2022-01-01 01:00:00+00:00,4.630245,0.0,3.259939,173.65980
2,2022-01-01 02:00:00+00:00,3.660245,0.0,3.545589,156.03750
3,2022-01-01 03:00:00+00:00,3.880245,0.0,3.671294,168.69006
4,2022-01-01 04:00:00+00:00,3.650245,0.0,4.349896,155.55605


In [21]:
Weather_full.to_csv("./Cleaned/Wetterdaten_Basel_cleaned22_24.csv")

In [22]:
Gas_full = pd.read_csv("./Gasverbrauch_im_Versorgungsgebiet_der_IWB.csv", sep=';')

In [35]:
Gas_full.head()
print(Gas_full[Gas_full["Datum"] == "2022-03-27 00:00:00+00:00"])
Gas_full[Gas_full["Datum"] == "2022-03-27 03:00:00+02:00"]


                          Datum  Gasverbrauch
23890 2022-03-27 00:00:00+00:00   227953.7917


,Datum,Gasverbrauch
23891,2022-03-27 01:00:00+00:00,251581.2461
3201,2022-03-27 01:00:00+00:00,346278.7600


In [24]:
Gas_full.sort_values(by="Start der Messung", inplace=True)
Gas_full.head()

Gas_full.drop(columns=["Datum", "Zeit", "Jahr", "Monat", "Tag", "Wochentag", "Tag des Jahres", "Quartal", "Woche des Jahres"], inplace=True)

Gas_full['Start der Messung'] = pd.to_datetime(Gas_full['Start der Messung'], utc=True)

Gas_full.rename(columns={'Start der Messung': 'Datum'}, inplace=True)
#Gas_full.set_index('Datum', inplace=True)
Gas_full = Gas_full[Gas_full['Datum'] >= '2022-01-01']
#Gas_full.index = Gas_full.index.str.replace('T', ' ')

In [31]:
Gas_full.head()

print(Gas_full[Gas_full["Datum"] == "2022-03-27 00:00:00+00:00"])
Gas_full[Gas_full["Datum"] == "2022-03-27 02:00:00+00:00"]

                          Datum  Gasverbrauch
23890 2022-03-27 00:00:00+00:00   227953.7917


,Datum,Gasverbrauch
23892,2022-03-27 02:00:00+00:00,438581.9031


In [127]:


#hopefully useless
#checking for missing values
Gas_full.reset_index(inplace=True)

# Create a date range from the minimum to the maximum date in the 'Datum' column
date_range = pd.date_range(start=Gas_full['Datum'].min(), end=Gas_full['Datum'].max(), freq='D')

# Create a DataFrame with all the dates in the range
all_dates = pd.DataFrame({'Datum': date_range})

# Merge this new DataFrame with the original to find missing dates
missing_dates = pd.merge(all_dates, Gas_full, on='Datum', how='left', indicator=True)

# Rows that are missing in the original DataFrame will have '_merge' == 'left_only'
missing_rows = missing_dates[missing_dates['_merge'] == 'left_only']

# Output the missing rows (if any)
if not missing_rows.empty:
    print(f"Missing rows for the following dates: {missing_rows['Datum'].tolist()}")
else:
    print("No rows are missing.")

Missing rows for the following dates: [Timestamp('2022-10-30 00:00:00+0000', tz='UTC'), Timestamp('2023-10-29 00:00:00+0000', tz='UTC'), Timestamp('2024-10-27 00:00:00+0000', tz='UTC')]


In [14]:
Gas_full.head()

,Datum,Gasverbrauch
4229,2022-01-01 00:00:00+00:00,344980.5901
4230,2022-01-01 01:00:00+00:00,357897.5343
25727,2022-01-01 02:00:00+00:00,363614.8815
4231,2022-01-01 03:00:00+00:00,411105.7247
13339,2022-01-01 04:00:00+00:00,512091.2476


In [129]:
Gas_full.index = Gas_full['Datum']

# Check for duplicates in the index
print(Gas_full.index.duplicated().sum())  # Number of duplicates
print(Gas_full.index[Gas_full.index.duplicated()])  # Which labels are duplicates

duplicates = Gas_full.index.duplicated(keep=False)
print(Gas_full[duplicates])



3
DatetimeIndex(['2022-03-27 01:00:00+00:00', '2023-03-26 01:00:00+00:00',
               '2024-03-31 01:00:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Datum', freq=None)
                           index                     Datum  Gasverbrauch
Datum                                                                   
2022-03-27 01:00:00+00:00  23891 2022-03-27 01:00:00+00:00   251581.2461
2022-03-27 01:00:00+00:00   3201 2022-03-27 01:00:00+00:00   346278.7600
2023-03-26 01:00:00+00:00  16823 2023-03-26 01:00:00+00:00   236378.3897
2023-03-26 01:00:00+00:00   6553 2023-03-26 01:00:00+00:00   254217.8493
2024-03-31 01:00:00+00:00   4705 2024-03-31 01:00:00+00:00   220161.4192
2024-03-31 01:00:00+00:00  26237 2024-03-31 01:00:00+00:00   244597.7287


In [28]:
print(Gas_full[Gas_full["Datum"] == "2022-03-27 00:00:00+00:00"])
Gas_full[Gas_full["Datum"] == "2022-03-27 02:00:00+00:00"]

                          Datum  Gasverbrauch
23890 2022-03-27 00:00:00+00:00   227953.7917


,Datum,Gasverbrauch
23892,2022-03-27 02:00:00+00:00,438581.9031


In [30]:
print(Gas_full[Gas_full["Datum"] == "2024-04-02 00:00:00+00:00"])
Gas_full[Gas_full["Datum"] == "2024-04-02 02:00:00+00:00"]

                         Datum  Gasverbrauch
7533 2024-04-02 00:00:00+00:00    238935.901


,Datum,Gasverbrauch
7534,2024-04-02 02:00:00+00:00,335333.973


In [29]:
print(Gas_full[Gas_full["Datum"] == "2023-03-26 00:00:00+00:00"])
Gas_full[Gas_full["Datum"] == "2023-03-26 02:00:00+00:00"]

                          Datum  Gasverbrauch
16822 2023-03-26 00:00:00+00:00   234133.2606


,Datum,Gasverbrauch
24103,2023-03-26 02:00:00+00:00,312927.9184


In [121]:
import pandas as pd

# Define the specific dates for which you want to apply the rolling mean
specific_dates = [
    pd.Timestamp('2022-10-30 00:00:00+0000', tz='UTC'),
    pd.Timestamp('2023-10-29 00:00:00+0000', tz='UTC'),
    pd.Timestamp('2024-10-27 00:00:00+0000', tz='UTC')
]

# Ensure 'Datum' is in datetime format (if not already)
#Gas_full.index = pd.to_datetime(Gas_full.index)

# Create a mask to filter the specific dates
mask = Gas_full.index.isin(specific_dates)

# Apply the rolling mean to the specified dates only
Gas_full.loc[mask, 'Gasverbrauch'] = Gas_full.loc[mask, 'Gasverbrauch'].fillna(
    Gas_full['Gasverbrauch'].rolling(window=3, min_periods=1).mean()
)

# Output the resulting DataFrame
print(Gas_full.loc[specific_dates])


ValueError: cannot reindex on an axis with duplicate labels

#standardize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# Reshape the column to a 2D array
Gas_full["Gasverbrauch_scaled"] = scaler.fit_transform(Gas_full["Gasverbrauch"].values.reshape(-1, 1))

# Verify the mean and standard deviation
Gas_full["Gasverbrauch_scaled"].mean(), Gas_full["Gasverbrauch_scaled"].std()


In [9]:

Gas_full.to_csv("./Cleaned/Gasverbrauch_cleaned.csv")

NameError: name 'Gas_full' is not defined